In [15]:
%load_ext autotime

In [16]:
import sys
sys.path.append('/scratch/pradap/python-work/enrique/')

time: 799 µs


In [17]:
import magellan as mg

time: 584 µs


In [18]:
import pandas as pd
import numpy as np

time: 697 µs


In [19]:
mg.init_jvm()

True

time: 1.78 ms


In [20]:
wal =  mg.read_csv(mg.get_install_path() + '/datasets/books/walmart.csv',
                    dtype={'isbn':pd.np.str, 'pages':pd.np.str, 'volume':pd.np.str, 'editionNum':pd.np.str},  
                    low_memory=False, key='id')


time: 955 ms


In [21]:
bwk = mg.read_csv(mg.get_install_path() + '/datasets/books/bowker.csv', 
                  dtype={'isbn':pd.np.str, 'pages':pd.np.str, 'volume':pd.np.str, 'editionNum':pd.np.str},  
                  low_memory=False, key='id')


time: 45.1 ms


In [22]:
ab = mg.AttrEquivalenceBlocker()
C = ab.block_tables(wal, bwk, 'isbn', 'isbn', ['title', 'author'], ['title', 'author'])


time: 110 ms


In [23]:
L = mg.read_csv('label_ab_correct_books.csv', ltable=wal, rtable=bwk)


time: 2.94 ms


In [24]:
feat_table = mg.get_features_for_matching(wal, bwk)
f = feat_table.ix[[3,7,18,26, 53]]

time: 2.6 s


In [25]:
G = mg.extract_feat_vecs(L, feat_table=f, attrs_after='gold')

time: 250 ms


In [46]:
dt = mg.DTMatcher(random_state=100)
svm = mg.SVMMatcher()
rf = mg.RFMatcher()
nb = mg.NBMatcher()
lg = mg.LogRegMatcher()

time: 1.77 ms


In [47]:
# impute values
G.fillna(0, inplace=True)

time: 1.07 ms


In [48]:
selected, stats = mg.select_matcher([dt, rf, svm, nb, lg], table=G, exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
                                  target_attr='gold', metric='precision')

time: 65.5 ms


In [49]:
stats

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,DecisionTree_69002655600081885441,<magellan.matcher.dtmatcher.DTMatcher object a...,5,0.875000,0.777778,0.909091,0.916667,0.833333,0.862374
1,RandomForest_44799765600081885441,<magellan.matcher.rfmatcher.RFMatcher object a...,5,0.857143,0.800000,0.900000,0.909091,0.833333,0.859913
2,SVM_23884265600081885441,<magellan.matcher.svmmatcher.SVMMatcher object...,5,0.500000,0.750000,0.888889,0.900000,0.833333,0.774444
3,NaiveBayes_84001475600081885441,<magellan.matcher.nbmatcher.NBMatcher object a...,5,0.875000,0.666667,0.875000,0.900000,1.000000,0.863333
4,LogisticRegression_65411875600081885441,<magellan.matcher.logregmatcher.LogRegMatcher ...,5,0.636364,0.727273,0.800000,0.900000,0.833333,0.779394


time: 18.6 ms


In [50]:
#d = mg.train_test_split(G, train_proportion=0.7)
train = mg.read_csv('train.csv', ltable=wal, rtable=bwk)
test = mg.read_csv('test.csv', ltable=wal, rtable=bwk)


time: 4.31 ms


In [42]:
dt.fit(table=train, exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'], target_attr='gold')

time: 4.98 ms


In [54]:
mg.vis_debug_dt(dt, train, test, exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
                                     target_attr='gold')

_id                                      int64
ltable.id                                int64
rtable.id                                int64
title_title_jac_qgm_3_qgm_3            float64
author_author_jac_qgm_3_qgm_3          float64
binding_binding_jac_qgm_3_qgm_3        float64
publisher_publisher_jac_qgm_3_qgm_3    float64
lang_lang_lev                          float64
gold                                     int64
_predicted                               int64
dtype: object
time: 28.6 s


In [ ]:
neg_trigger = 